# Point Process

In [1]:
import pandas as pd
import numpy as np
import theano
import pymc3 as pm
from scipy.stats import ks_2samp
import seaborn as sns
import matplotlib.pyplot as plt
import glob

sns.set_context('notebook')
plt.style.use('seaborn-darkgrid')
print('Running on PyMC3 v{}'.format(pm.__version__))
#theano.config.gcc.cxxflags = "-Wno-c++11-narrowing"
import warnings

Running on PyMC3 v3.9.1


## Dirichlet Mixture Model

In [2]:
def Bayesian_mixture_model(ISI_data):
    
    with pm.Model() as model:
        
        BoundedNormal = pm.Bound(pm.Normal, lower=0.0)
        mu1 = pm.Uniform('mu1',lower=0.01,upper=0.1)
        lam1 = pm.Uniform('lam1',lower=0.01,upper=0.04)
        obs1 = pm.Wald.dist(mu=mu1,lam=lam1)


        mu2 = pm.Uniform('mu2',lower=0,upper=0.2)
        sigma = pm.Uniform('sigma',lower=0.0001,upper=0.5)
        obs2 = pm.Bound(pm.Normal, lower=0.0).dist(mu=mu2,sigma=sigma)

        mu3 = pm.Uniform('mu3',lower=0.1,upper=0.6)
        sigma3 = pm.Uniform('sigma3',lower=0.0001,upper=0.5)
        obs3 = pm.Bound(pm.Normal, lower=0.0).dist(mu=mu3,sigma=sigma3)


        w = pm.Dirichlet('w', a=np.array([1., .4, .4]))

        like = pm.Mixture('like', w=w, comp_dists = [obs1, obs2, obs3], observed=ISI_data)


        #step = pm.Metropolis(target_accept=0.9)
        #trace = pm.sample(step=step,draws=500,tune=100,progressbar=False,random_seed=10,cores=6)
     

    map_estimate = pm.find_MAP(model=model)
    del map_estimate['w_stickbreaking__']
    del map_estimate['mu1_interval__']
    del map_estimate['lam1_interval__']
    del map_estimate['mu2_interval__']
    del map_estimate['sigma_interval__']
    del map_estimate['mu3_interval__']
    del map_estimate['sigma3_interval__']
    
    map_estimate['w1'] = map_estimate['w'][0]
    map_estimate['w2'] = map_estimate['w'][1]
    map_estimate['w3'] = map_estimate['w'][2]
    
    del map_estimate['w']
    
    map_estimate['n_spikes'] = ISI_data.shape[0]
    
    
    #with model:
    #    ppc_trace = pm.sample_posterior_predictive(trace, 50,model=model,progressbar=False,random_seed=10)
    
    #lista_samples=[]
    #for i in list(ppc_trace['like']):
    #    lista_samples.extend(i)
    
    
    #print('P_value: ',ks_2samp(lista_samples,ISI_data,mode = 'asymp').pvalue)
    
    return map_estimate

## Inverse Gaussian

In [3]:
def Inverse_Gaussian_model(ISI_data):    
    
    with pm.Model() as model:
        BoundedNormal = pm.Bound(pm.Normal, lower=0.0)
        mu1 = pm.Uniform('mu1',lower=0.01,upper=0.1)
        lam1 = pm.Uniform('lam1',lower=0.01,upper=0.04)
        obs1 = pm.Wald('like',mu=mu1,lam=lam1,observed=ISI_healthy)


        step = pm.NUTS(target_accept=0.9)
        trace = pm.sample(step=step,draws=1000,tune=1000,cores=6)

        map_estimate = pm.find_MAP(model=model)
        
    
    del map_estimate['mu1_interval__']
    del map_estimate['lam1_interval__']
    
    
    
    
    map_estimate['n_spikes'] = ISI_data.shape[0]
    
    
    with model:
        ppc_trace = pm.sample_posterior_predictive(trace, 200,model=model)
    
    lista_samples=[]
    for i in list(ppc_trace['like']):
        lista_samples.extend(i)
    
    
    print('P_value: ',ks_2samp(lista_samples,ISI_data,mode = 'asymp').pvalue)
    
    return map_estimate

## Import all neural net

In [4]:
list_dir=glob.glob("Data after SS/*.txt")

In [5]:
list_dir_ok = []
for net in list_dir:
    if 'After' in net:
        print(net,'ok')
        list_dir_ok.append(net)

Data after SS/AfterHealthy2019-01-24T15-38-41MiP2 24hour after KA.h5.txt ok
Data after SS/AfterHealthy2019-01-24T15-49-35Pop3 24hour after KA.h5.txt ok
Data after SS/After2018-11-28T11-42-06MIP2 Cont 24hrs after.h5.txt ok
Data after SS/After2018-11-28T12-22-45Pop1 control.h5.txt ok
Data after SS/After2018-11-28T11-29-05MIP 5 KA 24hrs after.h5.txt ok
Data after SS/AfterHealthy2018-10-13T15-42-42MIP5,  control, 24 hours post .h5.txt ok
Data after SS/After2018-11-28T11-55-53MIP3 KA 24hrs after.h5.txt ok
Data after SS/After2018-11-28T12-35-18Pop2 stimulation.h5.txt ok
Data after SS/After2018-11-28T12-08-45MIP4 KA 24hrs after.h5.txt ok
Data after SS/AfterHealthy2019-01-24T15-27-45MiP3 24hour after KA.h5.txt ok
Data after SS/After2018-11-28T12-47-25Pop3 Control.h5.txt ok
Data after SS/AfterHealthy2019-01-24T16-00-33Pop1 24hour after.h5.txt ok
Data after SS/AfterHealthy2018-10-13T15-10-36MIP2, control, 24 hours post.h5.txt ok


In [6]:
list_neurons = np.genfromtxt(net, delimiter=',')

## Run the model on the interval 100-300s

In [11]:
dataframe = pd.DataFrame()
counter_net=1
counter=0
target = 0
for net in list_dir_ok:
    print(counter_net,') ',net)
    counter_net+=1
    list_neurons = np.genfromtxt(net, delimiter=',')
    counter=0
    print('Original number of neurons: ',len(list_neurons))
    for neuron in list_neurons:
        neuron=neuron[neuron>0*10000]
        neuron=neuron[neuron<200*10000]
        print('  Neuron with ',neuron.shape[0],'spikes')
        if neuron.shape[0]>1000:
            
            counter+=1
        else:
            print('    Excluded neuron with n spikes = ',neuron.shape[0])
            continue
        
        ISI_healthy = np.diff(neuron)/10000
        
            
        map_estimate = Bayesian_mixture_model(ISI_healthy)
        
        if 'Healthy' in net:
            map_estimate['Target']=0
            target = 0
            
        elif 'Healthy' in net:
            map_estimate['Target']=0
            target = 0
        else:
            map_estimate['Target']=1
            target = 1
        
        if 'KA' in net:
            map_estimate['Stimulation']=0
            target = 0
            
        elif 'stimulation' in net:
            map_estimate['Stimulation']=0
            target = 0
        else:
            map_estimate['Stimulation']=1
            target = 1
        df = pd.DataFrame.from_dict(map_estimate,orient='index')
        dataframe = pd.concat([dataframe,df],axis = 1)
    print('Final number of neurons: ',counter)
    print('Target = ',target)

1 )  Data after SS/AfterHealthy2019-01-24T15-38-41MiP2 24hour after KA.h5.txt
Original number of neurons:  63
  Neuron with  132 spikes
    Excluded neuron with n spikes =  132
  Neuron with  5996 spikes



  Neuron with  168 spikes
    Excluded neuron with n spikes =  168
  Neuron with  6387 spikes



  Neuron with  196 spikes
    Excluded neuron with n spikes =  196
  Neuron with  6909 spikes



  Neuron with  148 spikes
    Excluded neuron with n spikes =  148
  Neuron with  6322 spikes



  Neuron with  358 spikes
    Excluded neuron with n spikes =  358
  Neuron with  4370 spikes


...


  Neuron with  481 spikes
    Excluded neuron with n spikes =  481
  Neuron with  595 spikes
    Excluded neuron with n spikes =  595
  Neuron with  509 spikes
    Excluded neuron with n spikes =  509
  Neuron with  163 spikes
    Excluded neuron with n spikes =  163
  Neuron with  171 spikes
    Excluded neuron with n spikes =  171
  Neuron with  177 spikes
    Excluded neuron with n spikes =  177
  Neuron with  176 spikes
    Excluded neuron with n spikes =  176
  Neuron with  202 spikes
    Excluded neuron with n spikes =  202
  Neuron with  175 spikes
    Excluded neuron with n spikes =  175
  Neuron with  208 spikes
    Excluded neuron with n spikes =  208
  Neuron with  224 spikes
    Excluded neuron with n spikes =  224
  Neuron with  218 spikes
    Excluded neuron with n spikes =  218
  Neuron with  261 spikes
    Excluded neuron with n spikes =  261
  Neuron with  268 spikes
    Excluded neuron with n spikes =  268
  Neuron with  220 spikes
    Excluded neuron with n spikes =

INFO (theano.gof.compilelock): Refreshing lock /home/riccardo_levi_25_gmail_com/.theano/compiledir_Linux-5.4--gcp-x86_64-with-glibc2.10-x86_64-3.8.3-64/lock_dir/lock



  Neuron with  220 spikes
    Excluded neuron with n spikes =  220
  Neuron with  483 spikes
    Excluded neuron with n spikes =  483
  Neuron with  440 spikes
    Excluded neuron with n spikes =  440
  Neuron with  989 spikes
    Excluded neuron with n spikes =  989
  Neuron with  512 spikes
    Excluded neuron with n spikes =  512
  Neuron with  2025 spikes



  Neuron with  299 spikes
    Excluded neuron with n spikes =  299
  Neuron with  240 spikes
    Excluded neuron with n spikes =  240
  Neuron with  419 spikes
    Excluded neuron with n spikes =  419
  Neuron with  672 spikes
    Excluded neuron with n spikes =  672
  Neuron with  537 spikes
    Excluded neuron with n spikes =  537
  Neuron with  1067 spikes



  Neuron with  460 spikes
    Excluded neuron with n spikes =  460
  Neuron with  119 spikes
    Excluded neuron with n spikes =  119
  Neuron with  268 spikes
    Excluded neuron with n spikes =  268
  Neuron with  426 spikes
    Excluded neuron with n spikes =  426
  Neuron with  172 spikes
    Excluded neuron with n spikes =  172
  Neuron with  212 spikes
    Excluded neuron with n spikes =  212
  Neuron with  173 spikes
    Excluded neuron with n spikes =  173
  Neuron with  174 spikes
    Excluded neuron with n spikes =  174
  Neuron with  273 spikes
    Excluded neuron with n spikes =  273
  Neuron with  170 spikes
    Excluded neuron with n spikes =  170
  Neuron with  189 spikes
    Excluded neuron with n spikes =  189
  Neuron with  228 spikes
    Excluded neuron with n spikes =  228
  Neuron with  188 spikes
    Excluded neuron with n spikes =  188
  Neuron with  167 spikes
    Excluded neuron with n spikes =  167
  Neuron with  196 spikes
    Excluded neuron with n spikes =

In [12]:
final = dataframe.T
final.to_csv('Data after PP/DataAfter')